# First load os and sys so I can update the sys.path with new functions

change the luminosity to 80 /fb

generate the 3 plots as in aewol paper.

In [1]:
import os
import sys

In [2]:
#take the paths to the functions we nedd
module_path = os.path.abspath(os.path.join('./pandasPlotting/'))
module2_path = os.path.abspath(os.path.join('./MlClasses/'))
module3_path = os.path.abspath(os.path.join('./MlFunctions/'))

In [3]:
# this part will include in the sys.path variables the paths for our new functions
if [module_path, module2_path, module3_path] not in sys.path:
    sys.path.append(module_path)

In [4]:
# here we are going to load what we will need, keras + tensorflow, plot functions, etc..
import matplotlib
matplotlib.use('Agg')
import pandas as pd
import numpy as np
import math
import time

from keras import callbacks

from pandasPlotting.Plotter import Plotter
from pandasPlotting.dfFunctions import expandArrays
from pandasPlotting.dtFunctions import featureImportance

from MlClasses.MlData import MlData
from MlClasses.Bdt import Bdt
from MlClasses.Dnn import Dnn
from MlClasses.ComparePerformances import ComparePerformances

from MlFunctions.DnnFunctions import significanceLoss,significanceLossInvert,significanceLoss2Invert ,significanceLossInvertSqrt,significanceFull,asimovSignificanceLoss,asimovSignificanceLossInvert,asimovSignificanceFull,truePositive,falsePositive

from linearAlgebraFunctions import gram,addGramToFlatDF

Using TensorFlow backend.


In [5]:
# I don't have patience to training 200 epochs ¯\_(ツ)_/¯
earlyStopping = callbacks.EarlyStopping(monitor='val_loss',min_delta=0,patience=5)

In [6]:
#load our data files
signal=pd.read_csv("../pyROOT_CPV_CPC/pp_wh/2dplots/analysis_with_cut/cpv_scan/charanjit_data/0L/feb/feb-0L-inclusive/zh0p0010lfeb.csv",sep='\s+',engine='python')
bkgd=pd.read_csv("../pyROOT_CPV_CPC/pp_wh/2dplots/analysis_with_cut/cpv_scan/charanjit_data/0L/feb/feb-0L-inclusive/zhsm0lfeb.csv",sep='\s+',engine='python')


In [7]:
#combine them into one dataset
combined = pd.concat([signal,bkgd]).sample(frac=1)

In [8]:
print(combined.keys())

Index(['ptb1', 'ptb2', 'misset', 'pth', 'ptz', 'etah', 'phih', 'mtvh', 'ptvh',
       'dphib1met', 'signal'],
      dtype='object')


In [9]:
# change thes vars depend on which dataset you are loading, I will implement a better solution.
chosenVars = {
            # #A vanilla analysis with HL variables and lead 3 jets
            '0L':['ptb1', 'ptb2', 'misset', 'pth', 'ptz', 'etah', 'phih', 'mtvh', 'ptvh', 'dphib1met', 'signal'],
            #'2L':['ptb1', 'ptb2', 'ptl1', 'ptl2', 'pth', 'ptz', 'etah', 'phih',
            #     'deltarll', 'deltarbl', 'mtvh', 'ptvh', 'dphil1b1', 'dphil1b2', 'signal']
}

In [10]:
trainedModels={}

In [11]:
#needed to plot asimov significane
asimovSigLossSysts=[0.01,0.05,0.1,0.2,0.3,0.4,0.5]

In [12]:
# here I have included one archtecture I got from my ES scan, pls comment my entry and use dnn_batch4096 instead.
dnnConfigs={
    #'dnn_ZH_0L_cHW_0d001_batch_4096':{'epochs':200,'batch_size':4096,'dropOut':None,
    #                                  'l2Regularization':None,'hiddenLayers':[1.0],
    #                                  'optimizer':'adam', 'activation':'relu'}
    #'dnn_ZH_0L_cHW_0d01_batch_4096':{'epochs':200,'batch_size':4096,'dropOut':None,
    #                                  'l2Regularization':None,'hiddenLayers':[1.0],
    #                                  'optimizer':'adam', 'activation':'relu'}
    'dnn_ZH_0L_cHW_0d03_batch_4096':{'epochs':200,'batch_size':4096,'dropOut':None,
                                      'l2Regularization':None,'hiddenLayers':[1.0],
                                      'optimizer':'adam', 'activation':'relu'}

}

In [13]:
#bsmlike signal cHW 0.03: 11.867fb, bg:0.89
#inclusive signal cHW 0.001: 2.198fb, bg:2.03fb
#inclusive signal cHW 0.01: 4.553fb, bg:2.03fb
#inclusive signal cHW 0.03: 14fb, bg:2.03fb

lumi=80. #luminosity in /fb
expectedSignal=2.198*lumi 
expectedBkgd=2.03*lumi #cross section of ttbar sample in fb times efficiency measured by Marco
systematic=0.5 #systematic for the asimov signficance

In [14]:
# I'm only running the DNN with binary cross entropy, letter I will include the other options.
# so far it running perfect, I'm testing with their custom loss function.
for varSetName,varSet in chosenVars.items():
    #Pick out the expanded arrays
    columnsInDataFrame = []
    for k in combined.keys():
        for v in varSet:
            #Little trick to ensure only the start of the string is checked
            if varSetName is '0L':
                if ' '+v+' ' in ' '+k+' ': columnsInDataFrame.append(k)
            elif ' '+v in ' '+k: columnsInDataFrame.append(k)


    #Select just the features we're interested in
    #For now setting NaNs to 0 for compatibility
    combinedToRun = combined[columnsInDataFrame].copy()
    combinedToRun.fillna(0,inplace=True)
    
    combinedToRun.index = np.arange(0,200000)
    mlData = MlData(combinedToRun,'signal')

    mlData.prepare(evalSize=0.0,testSize=0.3,limitSize=None)

    for name,config in dnnConfigs.items():
                        print 'Defining and fitting DNN with significance loss function',name
                        dnn = Dnn(mlData,'testPlots/mlPlots/sigLossInvert/'+varSetName+'/'+name)
                        dnn.setup(hiddenLayers=config['hiddenLayers'],dropOut=config['dropOut'],l2Regularization=config['l2Regularization'],
                                loss=significanceLossInvert(expectedSignal,expectedBkgd),
                                extraMetrics=[
                                    significanceLoss(expectedSignal,expectedBkgd),significanceFull(expectedSignal,expectedBkgd),
                                    asimovSignificanceFull(expectedSignal,expectedBkgd,systematic),truePositive,falsePositive
                                ])
                        dnn.fit(epochs=config['epochs'],batch_size=config['batch_size'],callbacks=[earlyStopping])
                        dnn.save()
                        print ' > Producing diagnostics'
                        dnn.diagnostics(batchSize=8192)
                        dnn.makeHepPlots(expectedSignal,expectedBkgd,asimovSigLossSysts,makeHistograms=makeHistograms)

trainedModels[varSetName+'_sigLossInvert_'+name]=dnn

/home/felipe/hepML/MlClasses/Dnn.py:101: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  self.history = self.model.fit(self.data.X_train.as_matrix(), self.data.y_train.as_matrix(), sample_weight=self.data.weights_train,
/home/felipe/hepML/MlClasses/Dnn.py:102: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  validation_data=(self.data.X_test.as_matrix(),self.data.y_test.as_matrix(),self.data.weights_test),


Train on 140000 samples, validate on 60000 samples
Epoch 1/200
140000/140000 [==============================] - 8s 60us/step - loss: -48.6713 - acc: 0.5050 - sigLoss: -48.6713 - significance: 6.9578 - asimovSignificance: 1.6675 - truePositive: 0.5463 - falsePositive: 0.5358 - val_loss: -65.2461 - val_acc: 0.5022 - val_sigLoss: -65.2461 - val_significance: 9.4029 - val_asimovSignificance: 1.6577 - val_truePositive: 0.9660 - val_falsePositive: 0.9635
Epoch 2/200
140000/140000 [==============================] - 0s 2us/step - loss: -72.4954 - acc: 0.4997 - sigLoss: -72.4954 - significance: 9.4226 - asimovSignificance: 1.6546 - truePositive: 0.9714 - falsePositive: 0.9712 - val_loss: -80.2756 - val_acc: 0.5010 - val_sigLoss: -80.2756 - val_significance: 9.5610 - val_asimovSignificance: 1.6548 - val_truePositive: 1.0000 - val_falsePositive: 1.0000
Epoch 3/200
140000/140000 [==============================] - 0s 2us/step - loss: -81.5198 - acc: 0.4994 - sigLoss: -81.5198 - significance: 9.5533

140000/140000 [==============================] - 0s 2us/step - loss: -91.2601 - acc: 0.4996 - sigLoss: -91.2601 - significance: 9.5610 - asimovSignificance: 1.6548 - truePositive: 1.0000 - falsePositive: 1.0000 - val_loss: -91.4133 - val_acc: 0.5010 - val_sigLoss: -91.4133 - val_significance: 9.5610 - val_asimovSignificance: 1.6548 - val_truePositive: 1.0000 - val_falsePositive: 1.0000
Epoch 42/200
140000/140000 [==============================] - 0s 2us/step - loss: -91.3000 - acc: 0.4996 - sigLoss: -91.3000 - significance: 9.5610 - asimovSignificance: 1.6548 - truePositive: 1.0000 - falsePositive: 1.0000 - val_loss: -91.4133 - val_acc: 0.5010 - val_sigLoss: -91.4133 - val_significance: 9.5610 - val_asimovSignificance: 1.6548 - val_truePositive: 1.0000 - val_falsePositive: 1.0000
Epoch 43/200
140000/140000 [==============================] - 0s 2us/step - loss: -91.3130 - acc: 0.4996 - sigLoss: -91.3130 - significance: 9.5610 - asimovSignificance: 1.6548 - truePositive: 1.0000 - falsePo

140000/140000 [==============================] - 0s 3us/step - loss: -91.4047 - acc: 0.4996 - sigLoss: -91.4047 - significance: 9.5610 - asimovSignificance: 1.6548 - truePositive: 1.0000 - falsePositive: 1.0000 - val_loss: -91.4135 - val_acc: 0.5010 - val_sigLoss: -91.4135 - val_significance: 9.5610 - val_asimovSignificance: 1.6548 - val_truePositive: 1.0000 - val_falsePositive: 1.0000
Epoch 82/200
140000/140000 [==============================] - 0s 3us/step - loss: -91.4089 - acc: 0.4996 - sigLoss: -91.4089 - significance: 9.5610 - asimovSignificance: 1.6548 - truePositive: 1.0000 - falsePositive: 1.0000 - val_loss: -91.4135 - val_acc: 0.5010 - val_sigLoss: -91.4135 - val_significance: 9.5610 - val_asimovSignificance: 1.6548 - val_truePositive: 1.0000 - val_falsePositive: 1.0000
Epoch 83/200
140000/140000 [==============================] - 0s 3us/step - loss: -91.4050 - acc: 0.4996 - sigLoss: -91.4050 - significance: 9.5610 - asimovSignificance: 1.6548 - truePositive: 1.0000 - falsePo

140000/140000 [==============================] - 0s 2us/step - loss: -91.4124 - acc: 0.4996 - sigLoss: -91.4124 - significance: 9.5610 - asimovSignificance: 1.6548 - truePositive: 1.0000 - falsePositive: 1.0000 - val_loss: -91.4135 - val_acc: 0.5010 - val_sigLoss: -91.4135 - val_significance: 9.5610 - val_asimovSignificance: 1.6548 - val_truePositive: 1.0000 - val_falsePositive: 1.0000
Epoch 122/200
140000/140000 [==============================] - 0s 3us/step - loss: -91.4126 - acc: 0.4996 - sigLoss: -91.4126 - significance: 9.5610 - asimovSignificance: 1.6548 - truePositive: 1.0000 - falsePositive: 1.0000 - val_loss: -91.4135 - val_acc: 0.5010 - val_sigLoss: -91.4135 - val_significance: 9.5610 - val_asimovSignificance: 1.6548 - val_truePositive: 1.0000 - val_falsePositive: 1.0000
Epoch 123/200
140000/140000 [==============================] - 0s 2us/step - loss: -91.4124 - acc: 0.4996 - sigLoss: -91.4124 - significance: 9.5610 - asimovSignificance: 1.6548 - truePositive: 1.0000 - false

140000/140000 [==============================] - 0s 3us/step - loss: -91.4132 - acc: 0.4996 - sigLoss: -91.4132 - significance: 9.5610 - asimovSignificance: 1.6548 - truePositive: 1.0000 - falsePositive: 1.0000 - val_loss: -91.4135 - val_acc: 0.5010 - val_sigLoss: -91.4135 - val_significance: 9.5610 - val_asimovSignificance: 1.6548 - val_truePositive: 1.0000 - val_falsePositive: 1.0000
Epoch 162/200
140000/140000 [==============================] - 0s 3us/step - loss: -91.4129 - acc: 0.4996 - sigLoss: -91.4129 - significance: 9.5610 - asimovSignificance: 1.6548 - truePositive: 1.0000 - falsePositive: 1.0000 - val_loss: -91.4135 - val_acc: 0.5010 - val_sigLoss: -91.4135 - val_significance: 9.5610 - val_asimovSignificance: 1.6548 - val_truePositive: 1.0000 - val_falsePositive: 1.0000
Epoch 163/200
140000/140000 [==============================] - 0s 2us/step - loss: -91.4133 - acc: 0.4996 - sigLoss: -91.4133 - significance: 9.5610 - asimovSignificance: 1.6548 - truePositive: 1.0000 - false

/home/felipe/hepML/MlClasses/Dnn.py:221: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  report = self.model.evaluate(X_test.as_matrix(), y_test.as_matrix(), sample_weight=weights_test, batch_size=batchSize)
/home/felipe/hepML/MlClasses/Dnn.py:227: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  classificationReport(self.model.predict_classes(X_test.as_matrix()),self.model.predict(X_test.as_matrix()),y_test,f)


140000/140000 [==============================] - 0s 1us/step


/home/felipe/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/felipe/hepML/MlClasses/Dnn.py:232: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  report = self.model.evaluate(X_train.as_matrix(), y_train.as_matrix(), sample_weight=weights_train, batch_size=batchSize)
/home/felipe/hepML/MlClasses/Dnn.py:236: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  classificationReport(self.model.predict_classes(X_train.as_matrix()),self.model.predict(X_train.as_matrix()),y_train,f)
/home/felipe/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted sample